# Natural Language Inference using Neural Networks
Adam Ek

----------------------------------

The lab is an exploration and learning exercise to be done in a group and also in discussion with the teachers and other students.

Before starting, please read the instructions on [how to work on group assignments](https://github.com/sdobnik/computational-semantics/blob/master/README.md).

Write all your answers and the code in the appropriate boxes below.

----------------------------------

In this lab we'll work with neural networks for natural language inference. Our task is: given a premise sentence P and hypothesis H, what entailment relationship holds between them? Is H entailed by P, contradicted by P or neutral towards P?

Given a sentence P, if H definitely describe something true given P then it is an **entailment**. If H describe something that's *maybe* true given P, it's **neutral**, and if H describe something that's definitely *false* given P it's a **contradiction**. 

# 1. Data

We will explore natural language inference using neural networks on the SNLI dataset, described in [1]. The dataset can be downloaded [here](https://nlp.stanford.edu/projects/snli/). We prepared a "simplified" version, with only the relevant columns [here](https://gubox.box.com/s/idd9b9cfbks4dnhznps0gjgbnrzsvfs4).

The (simplified) data is organized as follows (tab-separated values):
* Column 1: Premise
* Column 2: Hypothesis
* Column 3: Relation

Like in the previous lab, we'll use torchtext to build a dataloader. You can essentially do the same thing as you did in the last lab, but with our new dataset. **[1 mark]**

In [1]:
# first we import some packages that we need
import torch
import torch.nn as nn
import torchtext

# our hyperparameters (add more when/if you need them)
device = torch.device('cuda:0')

batch_size = 8
learning_rate = 0.001
epochs = 3

#other packages that we are going to use
import random
import math

import numpy as np
import pandas as pd

#### Download and unzip: https://github.com/sdobnik/computational-semantics/blob/master/assignments/05-natural-language-inference/simple_snli_1.0.zip

In [2]:
def dataloader(path_to_snli):
    # your code goes here
    field_premise = torchtext.data.Field(lower=True, batch_first=True)
    field_hypothesis = torchtext.data.Field(lower=True, batch_first=True)
    field_relation = torchtext.data.Field(lower=True, batch_first=True, pad_token=False)
    
    train_dataset, dev_dataset, test_dataset = torchtext.data.TabularDataset.splits(path_to_snli,
                                                   train = 'simple_snli_1.0_train.csv', 
                                                   validation = 'simple_snli_1.0_dev.csv',
                                                   test = 'simple_snli_1.0_test.csv',
                                                   format = 'csv', 
                                                   fields = [('premise', field_premise),
                                                    ('hypothesis', field_hypothesis),
                                                    ('relation', field_relation)],
                                                   csv_reader_params = {'delimiter':'\t', 'quotechar': '½'})
    # adding the csv_reader_params as per your POS tutorial caused issues, so we decided not to include that in this notebook.
    
    field_premise.build_vocab(train_dataset)
    field_hypothesis.build_vocab(train_dataset)
    field_relation.build_vocab(train_dataset)
    
    train_iterator, dev_iterator, test_iterator = torchtext.data.BucketIterator.splits(
                                                                        (train_dataset, dev_dataset, test_dataset), 
                                                                         batch_size = batch_size,
                                                                         sort=False,
                                                                         repeat=True, 
                                                                         shuffle=True, 
                                                                         device=device)
    
    return train_iterator, dev_iterator, test_iterator, field_premise, field_hypothesis, field_relation

In [3]:
train, dev, test, premises, hypotheses, relations = dataloader('./')

In [4]:
len(relations.vocab)

6

In [5]:
# I do not understand why the relations.vocab has the length of 6: as you can see below, there are only 4 relations + pad token
# (setting it to False in dataloader dropped the size to 5, but I still do not get what the last element is). I also cannot find
# a working way to check what is INSIDE the vocabulary, the methods of torchtext.vocab.Vocab do not work with this.
import csv
relation = set()
with open('simple_snli_1.0_dev.csv', newline='') as csvfile:
     spamreader = csv.reader(csvfile, delimiter='\t', quotechar='|')
     for row in spamreader:
        relation.add(row[2])
        
with open('simple_snli_1.0_test.csv', newline='') as csvfile:
     spamreader = csv.reader(csvfile, delimiter='\t', quotechar='|')
     for row in spamreader:
        relation.add(row[2])
        
with open('simple_snli_1.0_train.csv', newline='') as csvfile:
     spamreader = csv.reader(csvfile, delimiter='\t', quotechar='|')
     for row in spamreader:
        relation.add(row[2])
        
print(relation)

{'contradiction', 'neutral', '-', 'entailment'}


# 2. Model

In this part, we'll build the model for predicting the relationship between H and P.

We will process each sentence using an LSTM. Then, we will construct some representation of the sentence. When we have a representation for H and P, we will combine them into one vector which we can use to predict the relationship.

We will train a model described in [2], the BiLSTM with max-pooling model. The procedure for the model is roughly:

    1) Encode the Hypothesis and the Premise using one shared bidirectional LSTM (or two different LSTMS)
    2) Perform max over the tokens in the premise and the hypothesis
    3) Combine the encoded premise and encoded hypothesis into one representation
    4) Predict the relationship 

### Creating a representation of a sentence

Let's first consider step 2 where we perform max/mean pooling. There is a function in pytorch for this, but we'll implement it from scratch. 

Let's consider the general case, what we want to do for these methods is apply some function $f$ along dimension $i$, and we want to do this for all $i$'s. As an example we consider the matrix S with size ``(N, D)`` where N is the number of words and D the number of dimensions:

$S = \begin{bmatrix}
    s_{11} & s_{12} & s_{13} & \dots  & s_{1d} \\
    s_{21} & s_{22} & s_{23} & \dots  & s_{2d} \\
    \vdots & \vdots & \vdots & \ddots & \vdots \\
    s_{n1} & s_{n2} & s_{n3} & \dots  & s_{nd}
\end{bmatrix}$

What we want to do is apply our function $f$ on each dimension, taking the input $s_{1d}, s_{2d}, ..., s_{nd}$ and generating the output $x_d$. 

You will implement both the max pooling method. When performing max-pooling, $max$ will be the function which selects a _maximum_ value from a vector and $x$ is the output, thus for each dimension $d$ in our output $x$ we get:

\begin{equation}
    x_d = max(s_{1d}, s_{2d}, ..., s_{nd})
\end{equation}


This operation will reduce a batch of size ``(batch_size, num_words, dimensions)`` to ``(batch_size, dimensions)`` meaning that we now have created a sentence representation based on the content of the words representations in the sentence. 

Create a function that takes as input a tensor of size ``(batch_size, num_words, dimensions)`` then performs max pooling and returns the result (the output should be of size: ```(batch_size, dimensions)```). [**4 Marks**]

In [6]:
num_words = 3
dimensions = 5
# A tensor for testing
test_tensor = torch.rand([batch_size, num_words, dimensions], dtype=torch.float64, device=device)
print(test_tensor)

tensor([[[6.0018e-01, 9.3254e-01, 1.5991e-01, 5.4113e-01, 4.4711e-01],
         [1.8128e-01, 5.7962e-02, 8.4768e-01, 6.8903e-01, 3.0194e-01],
         [1.8579e-01, 6.8336e-01, 3.2044e-01, 8.5323e-01, 3.0906e-01]],

        [[2.6510e-02, 1.0137e-01, 8.2373e-01, 3.3546e-01, 1.8819e-01],
         [2.1763e-01, 2.1628e-01, 2.4722e-01, 6.7042e-01, 3.1387e-01],
         [9.9254e-01, 6.1488e-01, 7.0384e-01, 7.8223e-01, 1.4766e-01]],

        [[2.2560e-01, 9.4026e-01, 3.6296e-01, 3.3529e-02, 5.8864e-01],
         [9.2015e-01, 8.4432e-01, 4.7033e-01, 9.9194e-01, 6.3490e-01],
         [7.0116e-01, 5.0775e-01, 4.4205e-01, 8.1613e-01, 2.9596e-01]],

        [[6.1439e-01, 7.6977e-01, 4.4447e-01, 9.8468e-01, 9.3032e-01],
         [5.9445e-01, 5.2541e-02, 8.8084e-01, 3.3992e-01, 5.5538e-02],
         [1.5560e-01, 4.9737e-02, 5.3115e-01, 8.9348e-02, 8.8493e-01]],

        [[1.4212e-01, 1.8661e-01, 1.9368e-02, 2.4984e-01, 2.1182e-01],
         [9.1856e-04, 6.3179e-01, 4.9441e-01, 9.0327e-01, 9.1718e-01]

In [16]:
def pooling(input_tensor):
    batches = len(input_tensor)
    words = len(input_tensor[0])
    dims = len(input_tensor[0][0])

    new_tensors = []

    for i in range(0,batches):
        new_tensor = []

        for j in range(0,dims):
            temp_tensor = []
        
            for k in range(0,words):
                temp_tensor.append(input_tensor[i][k][j])
            
            max_val = max(temp_tensor)
            new_tensor.append(max_val)
    
        actual_new_tensor = new_tensor[0].unsqueeze(0)
    
        for l in range(1,len(new_tensor)):
            actual_new_tensor = torch.cat((actual_new_tensor, new_tensor[l].unsqueeze(0)))

        new_tensors.append(actual_new_tensor)
    
    output_tensor = torch.stack(new_tensors)

    return output_tensor

In [17]:
pooling(test_tensor)

tensor([[0.6002, 0.9325, 0.8477, 0.8532, 0.4471],
        [0.9925, 0.6149, 0.8237, 0.7822, 0.3139],
        [0.9201, 0.9403, 0.4703, 0.9919, 0.6349],
        [0.6144, 0.7698, 0.8808, 0.9847, 0.9303],
        [0.2678, 0.7523, 0.4944, 0.9033, 0.9172],
        [0.4276, 0.8344, 0.5877, 0.9342, 0.4552],
        [0.3710, 0.6747, 0.8255, 0.7800, 0.8585],
        [0.3989, 0.8211, 0.7593, 0.7728, 0.9757]], device='cuda:0',
       dtype=torch.float64)

### Combining sentence representations

Next, we need to combine the premise and hypothesis into one representation. We will do this by concatenating four tensors (the final size of our tensor $X$ should be ``(batch_size, 4d)`` where ``d`` is the number of dimensions that you use): 

$$X = [P; H; |P-H|; P \cdot H]$$

Here, what we do is concatenating P, H, P times H, and the absolute value of P minus H, then return the result.

Implement the function. **[2 marks]**

In [19]:
# Test tensors (size of batch, num, dim)
t = torch.rand([2*batch_size, num_words, dimensions], dtype=torch.float64, device=device)
t1, t2 = torch.split(t, batch_size)
# Pooled test tensors (size of batch, dim)
pt1 = pooling(t1)
pt2 = pooling(t2)
print(pt1)
print(pt2)

tensor([[0.7285, 0.5180, 0.6645, 0.9100, 0.9208],
        [0.9668, 0.9276, 0.4489, 0.7350, 0.8521],
        [0.7451, 0.9273, 0.6983, 0.8307, 0.8093],
        [0.8488, 0.6891, 0.7451, 0.9290, 0.5603],
        [0.9258, 0.8266, 0.5683, 0.6003, 0.1685],
        [0.4605, 0.6107, 0.6603, 0.6723, 0.1351],
        [0.9333, 0.6884, 0.6262, 0.6547, 0.6435],
        [0.3485, 0.7087, 0.6973, 0.8247, 0.7439]], device='cuda:0',
       dtype=torch.float64)
tensor([[0.8100, 0.7544, 0.9747, 0.9113, 0.9875],
        [0.6869, 0.9990, 0.9677, 0.9382, 0.8886],
        [0.9550, 0.4418, 0.3069, 0.9933, 0.6600],
        [0.9891, 0.9923, 0.8862, 0.4801, 0.8949],
        [0.8439, 0.9095, 0.9705, 0.9383, 0.8354],
        [0.6751, 0.8652, 0.3123, 0.5073, 0.4134],
        [0.7555, 0.7993, 0.2765, 0.8148, 0.7798],
        [0.9715, 0.6143, 0.4070, 0.9172, 0.7147]], device='cuda:0',
       dtype=torch.float64)


In [29]:
def combine_premise_and_hypothesis(hypothesis, premise):
    
    batches = len(hypothesis)
    dims = len(hypothesis[0])
    final_dims = 4*dims

    new_tensors = []

    for i in range(0,batches):
        hyp = hypothesis[i]
        pre = premise[i]
    
        summed = torch.cat((pre,hyp))
        subtracted = pre - hyp
        multiplied = torch.mul(pre, hyp)
    
        new_tensors.append(torch.cat((summed, subtracted, multiplied)))
    
    output = torch.stack(new_tensors)
    
    return output

In [30]:
combine_premise_and_hypothesis(pt1, pt2)

tensor([[ 0.8100,  0.7544,  0.9747,  0.9113,  0.9875,  0.7285,  0.5180,  0.6645,
          0.9100,  0.9208,  0.0815,  0.2364,  0.3102,  0.0013,  0.0667,  0.5901,
          0.3908,  0.6477,  0.8293,  0.9092],
        [ 0.6869,  0.9990,  0.9677,  0.9382,  0.8886,  0.9668,  0.9276,  0.4489,
          0.7350,  0.8521, -0.2799,  0.0714,  0.5188,  0.2032,  0.0366,  0.6641,
          0.9267,  0.4344,  0.6896,  0.7571],
        [ 0.9550,  0.4418,  0.3069,  0.9933,  0.6600,  0.7451,  0.9273,  0.6983,
          0.8307,  0.8093,  0.2099, -0.4856, -0.3914,  0.1626, -0.1493,  0.7115,
          0.4097,  0.2143,  0.8251,  0.5341],
        [ 0.9891,  0.9923,  0.8862,  0.4801,  0.8949,  0.8488,  0.6891,  0.7451,
          0.9290,  0.5603,  0.1404,  0.3033,  0.1410, -0.4490,  0.3347,  0.8396,
          0.6838,  0.6603,  0.4460,  0.5014],
        [ 0.8439,  0.9095,  0.9705,  0.9383,  0.8354,  0.9258,  0.8266,  0.5683,
          0.6003,  0.1685, -0.0818,  0.0829,  0.4023,  0.3380,  0.6670,  0.7813,
      

### Creating the model

Finally, we can build the model according to the procedure given previously by using the functions we defined above. Additionaly, in the model you should use *dropout*. For efficiency purposes, it's acceptable to only train the model with either max or mean pooling. 

Implement the model [**6 marks**]

In [ ]:
class SNLIModel(nn.Module):
    def __init__(self, ...):
        # your code goes here
        self.embeddings = ...
        self.rnn = ...
        self.classifier = ...
        
    def forward(self, premise, hypothesis):
        p = ...
        h = ...
        
        p_pooled = pooling(...)
        h_pooled = pooling(...)
        
        ph_representation = combine_premise_and_hypothesis(...)
        predictions = ...
        
        return predictions

# 3. Training and testing

As before, implement the training and testing of the model. SNLI can take a very long time to train, so I suggest you only run it for one or two epochs. **[2 marks]** 

**Tip for efficiency:** *when developing your model, try training and testing the model on one batch (for each epoch) of data to make sure everything works! It's very annoying if you train for N epochs to find out that something went wrong when testing the model, or to find that something goes wrong when moving from epoch 0 to epoch 1.*

In [ ]:
train_iter, test_iter = dataloader(path_to_folder)

loss_function = ...
optimizer = ...
model = ...

for _ in range(epochs):
    # train model
    ...
    
# test model after all epochs are completed

Suggest a _baseline_ that we can compare our model against **[2 marks]**

**Your answer should go here**

Suggest some ways (other than using a baseline) in which we can analyse the models performance **[4 marks]**.

**Your answer should go here**

Suggest some ways to improve the model **[3 marks]**.

**Your answer should go here**

### Readings

[1] Samuel R. Bowman, Gabor Angeli, Christopher Potts, and Christopher D. Manning. 2015. A large annotated corpus for learning natural language inference. In Proceedings of the 2015 Conference on Empirical Methods in Natural Language Processing (EMNLP). 

[2] Conneau, A., Kiela, D., Schwenk, H., Barrault, L., & Bordes, A. (2017). Supervised learning of universal sentence representations from natural language inference data. arXiv preprint arXiv:1705.02364.